In [1]:
#import modules
import json
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv
from envs import env
import time
import re
import os
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
#import classes from ./ folder
import postgresdbAccess

In [3]:
#read fibonacci retracements  from json
with open('fibLvl.json') as file:
    fibLvl = json.load(file)

In [4]:
#load dotenv variables
load_dotenv('../.envall')
try:
    liveVolume=env("liveVolume")
    apiSecret=env('apiSecret')
    apiKey=env('apiKey')
    dbTable="table001"#env('dbTable')
    baseCurrency=env('baseCurrency')
    brokerFees=float(env('brokerFees'))
    liveTrading=env("liveTrading", 'False').lower() in ('true', '1', 't')
except KeyError:
    print("No env variables set.")
    sys.exit(1)
client = Client(apiKey, apiSecret, {'timeout':600})

In [17]:
#initiate postgresdb class
postgres = postgresdbAccess.postgresAccess()
sql = ("SELECT id, time, symbol, askprice, bidprice, stopid" +
        " FROM " + dbTable + " WHERE" +
        " resultpercent is not null order by id;")
decisions = pd.DataFrame(postgres.sqlQuery(sql))
decisions[0] = pd.to_numeric(decisions[0])
decisions[3] = pd.to_numeric(decisions[3])
decisions[1] = pd.to_datetime(decisions[1])
#initialize empty result list
result = []
for index, row in decisions.iterrows():
    #get row where trade was closed
    sql = ("SELECT askprice, bidprice" +
        " FROM " + dbTable + " WHERE" +
        " id = '" + str(int(row[5])) + "';")
    closeprice = pd.DataFrame(postgres.sqlQuery(sql))
    #realistic settle
    result.append(((closeprice[1][0] - row[3]) / row[3]) * 100 - brokerFees * 2.55)
print(sum(result) / len(result))
print(len(result))

0.1257455636404442
9
